In [1]:
import os
import pickle
import argparse
import numpy as np
from model import Options, Seq2SeqAttn


In [2]:
# Parse the command line arguments.
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type = str, default = './pre-data/',
                    help = 'the directory to the data')

parser.add_argument('--word_embeddings_path', type = str, default = './pre-data/word_embeddings.npy',
                    help = 'the directory to the pre-trained word embeddings')
parser.add_argument('--num_epochs', type = int, default = 10,
                    help = 'the number of epochs to train the data')
parser.add_argument('--batch_size', type = int, default = 1,
                    help = 'the batch size')
parser.add_argument('--learning_rate', type = float, default = 0.001,
                    help = 'the learning rate')
parser.add_argument('--beam_width', type = int, default = 256,
                    help = 'the beam width when decoding')
parser.add_argument('--word_embed_size', type = int, default = 256,
                    help = 'the size of word embeddings')
parser.add_argument('--n_hidden_units_enc', type = int, default = 256,
                    help = 'the number of hidden units of encoder')
parser.add_argument('--n_hidden_units_dec', type = int, default = 256,
                    help = 'the number of hidden units of decoder')
parser.add_argument('--attn_depth', type = int, default = 128,
                    help = 'attention depth')
parser.add_argument('--restore_path', type = str, default = './model_dailydialog_rf',
                    help = 'the path to restore the trained model')
parser.add_argument('--restore_epoch', type = int, default = 2,
                    help = 'the epoch to restore')
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
def read_data(data_path):
    def load_np_files(path):
        my_set = {}
        my_set['enc_input'] = np.load(os.path.join(path, 'enc_input.npy'))
        my_set['dec_input'] = np.load(os.path.join(path, 'dec_input.npy'))
        my_set['target'] = np.load(os.path.join(path, 'target.npy'))
        my_set['enc_input_len'] = np.load(os.path.join(path, 'enc_input_len.npy'))
        my_set['dec_input_len'] = np.load(os.path.join(path, 'dec_input_len.npy'))
        return my_set
    test_set = load_np_files(os.path.join(data_path, 'test'))
    # dictionary index of words
    with open(os.path.join(data_path, 'token2id.pickle'), 'rb') as file:
        token2id = pickle.load(file)
    with open(os.path.join(data_path, 'id2token.pickle'), 'rb') as file:
        id2token = pickle.load(file)
    return test_set, token2id, id2token


In [4]:
def ids_to_sentence(ids, uttr_len, id2token): # ?
    tokens = []
    if uttr_len is not None:
        for i in range(uttr_len):
            if id2token[ids[i]] != '<eos>' and id2token[ids[i]] != '<go>':
                tokens.append(id2token[ids[i]])
    else:
        i = 0
        while i < len(ids) and id2token[ids[i]] != '<eos>':
            tokens.append(id2token[ids[i]])
            i += 1
    return ' '.join(tokens)


In [5]:
if __name__ == '__main__':
    test_set, token2id, id2token = read_data(args.data_path)
    max_uttr_len_enc = test_set['enc_input'].shape[1]
    max_uttr_len_dec = test_set['dec_input'].shape[1]

    word_embeddings = np.load(args.word_embeddings_path)

    options = Options(mode = 'PREDICT',
                      num_epochs = args.num_epochs,
                      batch_size = args.batch_size,
                      learning_rate = args.learning_rate,
                      beam_width = args.beam_width,
                      vocab_size = len(token2id),
                      max_uttr_len_enc = max_uttr_len_enc,
                      max_uttr_len_dec = max_uttr_len_dec,
                      go_index = token2id['<go>'],
                      eos_index = token2id['<eos>'],
                      word_embed_size = args.word_embed_size,
                      n_hidden_units_enc = args.n_hidden_units_enc,
                      n_hidden_units_dec = args.n_hidden_units_dec,
                      attn_depth = args.attn_depth,
                      word_embeddings = word_embeddings)
    model = Seq2SeqAttn(options)

    for var in model.tvars:
        print(var.name)

    model.restore(os.path.join(args.restore_path, 'model_epoch_{:03d}.ckpt'.format(args.restore_epoch)))
    prediction = model.predict(test_set['enc_input'], test_set['enc_input_len'])

    pred_S2S = []
    f = open('./pre-data/test/pred_S2S.txt', 'w', encoding = 'utf-8')
    N = (test_set['enc_input'].shape[0] // args.batch_size) * args.batch_size
    for i in range(N):
        f.write('HISTORY:\n')
        uttr = ids_to_sentence(test_set['enc_input'][i,:], test_set['enc_input_len'][i], id2token)
        f.write('- {}\n'.format(uttr))
        f.write('LABEL:\n')
        label = ids_to_sentence(test_set['target'][i,:], test_set['dec_input_len'][i], id2token)
        f.write('- {}\n'.format(label))
        f.write('PREDICTION:\n')
        pred = ids_to_sentence(prediction[i//args.batch_size][i%args.batch_size,:,0], None, id2token)
        f.write('- {}\n\n'.format(pred))
        pred_S2S.append(pred)
    f.close()
    with open('./pre-data/test/pred_S2S.pickle', 'wb') as f:
        pickle.dump(pred_S2S, f)


Building the TensorFlow graph...
embedding/word_embeddings:0
encoding/rnn/gru_cell/gates/kernel:0
encoding/rnn/gru_cell/gates/bias:0
encoding/rnn/gru_cell/candidate/kernel:0
encoding/rnn/gru_cell/candidate/bias:0
decoding/memory_layer/kernel:0
decoding/attention_v:0
decoding/my_bahdanau_attention/query_layer/kernel:0
decoding/attention_wrapper/gru_cell/gates/kernel:0
decoding/attention_wrapper/gru_cell/gates/bias:0
decoding/attention_wrapper/gru_cell/candidate/kernel:0
decoding/attention_wrapper/gru_cell/candidate/bias:0
decoding/dense/kernel:0
decoding/dense/bias:0
Restoring a pre-trained model from ./model_dailydialog_rf/model_epoch_002.ckpt...
INFO:tensorflow:Restoring parameters from ./model_dailydialog_rf/model_epoch_002.ckpt


In [6]:
pred_S2S

['You got a warrant ?',
 'I owe you a celebration dinner .',
 "I can n't explain .",
 'Sure .',
 'Sure .',
 'Yeah .',
 "Psst ! It 's <NUM> .",
 "It 's okay .",
 'Well , throw me a fricking bone here , Scott .',
 'No , thanks .',
 'Right .',
 'Sure .',
 "Look , I 'm Zippi Longstocking .",
 'Answer me !',
 "It 's okay .",
 'Yeah .',
 "I can n't say now .",
 'What are you doing ?',
 "What 's that ?",
 'I hate vegetables .',
 "YOU READY ? LET 'S FIGHT .",
 'Ivana .',
 'No , sir .',
 "Have n't seen him since Friday .",
 'Get back to work .',
 'Again ?',
 'Huh ?',
 'Yeah .',
 'Again ?',
 'Yeah .',
 'Sorry .',
 'Yeah .',
 'What ?',
 'Sure .',
 'lf you wanted',
 'Thank you .',
 "YOU READY ? LET 'S FIGHT .",
 'You can relax here .',
 'I was in the city .',
 'YOU FIGHT .',
 'This is fucking wonderful .',
 'I hate vegetables .',
 'You can relax here .',
 'No , thanks .',
 "It 's-It 's King .",
 "What 's that ?",
 'Yeah .',
 'Great .',
 'Again ?',
 'Another bus wil come right ..',
 'Sorry .',
 'Yo